In [17]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pickle
from mutation import *
from crossover import *
from rmp import *
from mfea import *
from task import *
from llm import *
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# load_dotenv()

# GPT_API_KEY = os.getenv("GPT_API_KEY")

# llm = GPTModel(GPT_API_KEY, "gpt-3.5-turbo-0125", 0.7)

In [19]:
cec2017_tasks = get_10_tasks()

In [20]:
num_indis_per_task = 100
indi_len = 50
mutation = PolynomialMutation(5, 0.02)
crossover = SBXCrossover(mutation, eta=2)
# crossover = BLXCrossover()
rmp = AdaptiveRMPMatrix(rmp_pop_size=4, num_gen=3, pc=0.8, pm=0.1)

np.random.seed(0)
amfea = AMFEA(num_indis_per_task, indi_len, cec2017_tasks, crossover, mutation, rmp)

Initialization:
Task 0:
Best Fitness: 125473.96699307997
Mean Fitness: 166392.121167927

Task 1:
Best Fitness: 319204.0909081603
Mean Fitness: 503987.30720511527

Task 2:
Best Fitness: 346653.31463481847
Mean Fitness: 486737.256905658

Task 3:
Best Fitness: 41.06041433751163
Mean Fitness: 49.71421741800349

Task 4:
Best Fitness: 4058761661.357497
Mean Fitness: 6371677088.876898

Task 5:
Best Fitness: 21.482750101043788
Mean Fitness: 21.715487176250566

Task 6:
Best Fitness: 86.24596181476284
Mean Fitness: 99.85596727380062

Task 7:
Best Fitness: 18243.689210172182
Mean Fitness: 21153.768897608556

Task 8:
Best Fitness: 74.8950497645563
Mean Fitness: 122.64795239117853

Task 9:
Best Fitness: 88658.75615256254
Mean Fitness: 124389.07711105635



In [21]:
bfs, mfs = amfea.fit(num_gen=1000,monitor=True, monitor_rate=10, llm_rate=100)

Creating strategy
Creating strategy
Creating strategy
Creating strategy
Evaluating strategy
Strategy: 
Calculate the Euclidean distance between the mean vectors of each pair of tasks. Adjust RMP values inversely proportional to the distance, with closer tasks having higher mating probabilities.
Analyze the variance matrices of each task to determine the spread of individuals within the population. Increase RMP values between tasks with higher variances to promote cross-task exploration.
Compute the cosine similarity between the mean vectors of task pairs to assess the direction of distribution in the search space. Adjust RMP values based on the angle between vectors, with more aligned tasks having higher mating probabilities.
Evaluate the correlation coefficients between the mean vectors of different tasks to measure the strength of linear relationships. Modify RMP values according to the correlation strength, with positively correlated tasks having higher mating probabilities.
Creatin

d:\ANACONDA\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


[[1.         0.21666519 0.2314021  0.25525743 0.30844716 0.18606658
  0.20004332 0.22838384 0.19220727 0.2455072 ]
 [0.21666519 1.         0.2865571  0.27046454 0.26722549 0.16213515
  0.20807856 0.19349543 0.21787639 0.19910831]
 [0.2314021  0.2865571  1.         0.23934296 0.33518799 0.19556196
  0.24083718 0.2680264  0.22453082 0.24705401]
 [0.25525743 0.27046454 0.23934296 1.         0.35647333 0.18874687
  0.24320809 0.23913295 0.17320746 0.19709726]
 [0.30844716 0.26722549 0.33518799 0.35647333 1.         0.31540276
  0.27958682 0.30613834 0.30034034 0.28960285]
 [0.18606658 0.16213515 0.19556196 0.18874687 0.31540276 1.
  0.21325229 0.20433077 0.16799087 0.15601787]
 [0.20004332 0.20807856 0.24083718 0.24320809 0.27958682 0.21325229
  1.         0.26987719 0.2032936  0.26692611]
 [0.22838384 0.19349543 0.2680264  0.23913295 0.30613834 0.20433077
  0.26987719 1.         0.25075219 0.26815863]
 [0.19220727 0.21787639 0.22453082 0.17320746 0.30034034 0.16799087
  0.2032936  0.25075

<string>:7: RuntimeWarning: divide by zero encountered in divide


RMP_Matrix:
[[1.00000000e+00 1.22759832e-03 1.71697161e-03 4.20519995e-04
  4.15707723e-03 8.48275703e-04 4.21382464e-04 1.94180858e-03
  1.53498276e-03 3.48377987e-03]
 [1.22759832e-03 1.00000000e+00 6.95445842e-04 2.36301175e-04
  8.22337934e-04 4.54886366e-04 2.26247139e-04 8.80780598e-04
  6.30018972e-04 1.31905592e-03]
 [1.71697161e-03 6.95445842e-04 1.00000000e+00 4.99293323e-04
  2.35494149e-03 6.03869138e-04 3.89065810e-04 1.30352347e-03
  1.49195405e-03 1.17982142e-03]
 [4.20519995e-04 2.36301175e-04 4.99293323e-04 1.00000000e+00
  9.67120889e-06 2.21353366e-05 9.95922208e-06 1.13256679e-05
  1.28323164e-05 9.74729575e-06]
 [4.15707723e-03 8.22337934e-04 2.35494149e-03 9.67120889e-06
  1.00000000e+00 1.27554287e-03 7.00235206e-04 4.18799726e-03
  3.03626778e-03 4.33876047e-03]
 [8.48275703e-04 4.54886366e-04 6.03869138e-04 2.21353366e-05
  1.27554287e-03 1.00000000e+00 1.94688326e-03 1.51493254e-04
  7.13263991e-04 1.53671725e-04]
 [4.21382464e-04 2.26247139e-04 3.89065810e-04

...

RMP_Matrix:
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


<string>:13: RuntimeWarning: invalid value encountered in scalar divide


RMP_Matrix:
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Evaluating strategy
Strategy: 
Calculate the Euclidean distance between tasks' population mean vectors. Adjust RMP values based on the distance to promote mating between tasks that are close in the search space.
Analyze the cosine similarity between tasks' population mean vectors. Increase RMP values for task pairs with higher similarity to encourage mating between tasks exploring in similar directions.
Cluster tasks based on the Manhattan distance between population variance vectors. Adjust RMP values higher for task pairs within the same cluster to promote mating between tasks with similar diversity levels.
Assess the Mahalanobis distance between 

d:\ANACONDA\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\ANACONDA\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\ANACONDA\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\ANACONDA\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than availabl

Evaluating strategy
Strategy: 
Assess the Euclidean distance between tasks' population mean vectors. Decrease RMP values for task pairs with higher distance to discourage mating between tasks exploring in different directions.
Evaluate the covariance matrix between tasks' population variance vectors. Adjust RMP values based on the covariance to discourage mating between tasks with orthogonal spread in the search space.
Cluster tasks based on the DBSCAN algorithm using both pop_mean and pop_variance. Adjust RMP values higher for task pairs within the same cluster to encourage mating between tasks with similar exploration capabilities and density.
Calculate the Kullback-Leibler divergence between tasks' population mean and variance distributions. Adjust RMP values based on the divergence to discourage mating between tasks with significantly different distribution patterns.
Creating code...
RMP function: import numpy as np
from sklearn.cluster import DBSCAN
from scipy.stats import entropy

In [22]:
# num_tasks = len(tasks)
# fig, axes = plt.subplots(num_tasks, 2)
# fig.tight_layout()
# for i in range(num_tasks):
#     axes[i][0].plot(bfs[i])
#     axes[i][1].plot(mfs[i])